# PySpark NLP Tutorial

Upload this jupyter notebook to Google drive, and open this tutorial with Google Colab

In [0]:
# install the dependencies:
%env spark_version=2.4.4

!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-${spark_version}/spark-${spark_version}-bin-hadoop2.7.tgz
!tar xf spark-${spark_version}-bin-hadoop2.7.tgz
!pip install -q findspark
!python --version

In [0]:
# set environment
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-{}-bin-hadoop2.7".format(os.environ["spark_version"])

In [0]:
!cat /etc/spark-defaults.conf

In [0]:
# install pyspark
!pip install pyspark


In [0]:
import findspark
findspark.init() # os.environ["SPARK_HOME"]
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").getOrCreate()

In [0]:
# download data
import requests

url = "https://github.com/liuhoward/teaching/raw/master/big_data/smsspam/"
train_file = "SMSSpamCollection.train"
test_file = "SMSSpamCollection.test"

r = requests.get(url + train_file)
open(train_file, 'wb').write(r.content)

r = requests.get(url + test_file)
open(test_file, 'wb').write(r.content)


In [0]:
# load train data

from pyspark.sql.types import *

# define schema
data_schema = StructType([
    StructField("category", StringType()),
    StructField("text", StringType())])

# read train csv file
train_data = spark.read.csv(train_file, schema=data_schema, sep='\t', header=None)
print(type(train_data))

In [0]:
train_data.printSchema() # print detail schema of data
train_data.show(n=5, truncate=False) # show top 5 rows
train_data.count()  # number of examples


In [0]:
# lower case

from pyspark.sql.functions import lower, col

lower_train = train_data.select('category', lower(col('text')).alias('text'))

lower_train.show(n=5)

In [0]:
# tokenize

from pyspark.ml.feature import Tokenizer, RegexTokenizer
from pyspark.sql.functions import col, udf


tokenizer = Tokenizer(inputCol="text", outputCol="words")

tokenized_train = tokenizer.transform(lower_train).select("category", "words")


tokenized_train.show(n=5, truncate=False)

In [0]:
# remove stopwords

from pyspark.ml.feature import StopWordsRemover

remover = StopWordsRemover(inputCol="words", outputCol="filtered")
filtered_train = remover.transform(tokenized_train).select("category", "filtered")

filtered_train.show(n=5)

In [0]:
# convert feature to vector

from pyspark.ml.feature import CountVectorizer

# fit a CountVectorizerModel from the corpus.
vectorizer = CountVectorizer(inputCol="filtered", outputCol="features", vocabSize=10000, minDF=5.0)

cv_model = vectorizer.fit(filtered_train)

train_feature = cv_model.transform(filtered_train).select("category","features")
train_feature.show(n=5, truncate=False)

In [0]:
# convert label to index

from pyspark.ml.feature import StringIndexer

indexer = StringIndexer(inputCol="category", outputCol="label")

index_model = indexer.fit(train_feature)

train_xy = index_model.transform(train_feature).select("features", "label")

train_xy.show(n=5)


In [0]:
# preprocess test data

test_data = spark.read.csv(test_file, schema=data_schema, sep='\t', header=None)

lower_test = test_data.select('category', lower(col('text')).alias('text'))

tokenized_test = tokenizer.transform(lower_test).select("category", "words")

filtered_test = remover.transform(tokenized_test).select("category", "filtered")

test_feature = cv_model.transform(filtered_test).select("category","features")

test_xy = index_model.transform(test_feature).select("features", "label")

test_xy.show(n=5)


In [0]:
from pyspark.ml.classification import LogisticRegression


cls = LogisticRegression()

lrModel = cls.fit(train_xy)

In [0]:
predictions = lrModel.transform(test_xy)

predictions.show(n=5, truncate=False)

In [0]:
# evaluate AUC

from pyspark.ml.evaluation import BinaryClassificationEvaluator

evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction")

evaluator.evaluate(predictions, {evaluator.metricName: "areaUnderROC"})

In [0]:
# evaluate

from pyspark.mllib.evaluation import MulticlassMetrics

predictionAndLabels = predictions.select("prediction", "label").rdd
metrics = MulticlassMetrics(predictionAndLabels)


print(f"accuracy: {metrics.accuracy}")
print(f"precision: {metrics.precision(1.0)}")

print(f"recall: {metrics.recall(1.0)}")

print(f"f1 score: {metrics.fMeasure(1.0, 1.0)}")

In [0]:
# data is imbalanced

train_label_count = train_xy.groupby('label').count().toPandas()
train_label_count

In [0]:
# add weight to handle imbalanced classes

ratio = train_label_count.loc[0, 'count'] / train_label_count.loc[1, 'count']

ratio

In [0]:
from pyspark.sql.functions import when

def weight_balance(labels):
    return when(labels == 1, ratio).otherwise(1)

train_xy_weight = train_xy.withColumn('weights', weight_balance(col('label')))

train_xy_weight.show(n=5)

In [0]:
cls = LogisticRegression(weightCol="weights")

lrModel = cls.fit(train_xy_weight)

predictions = lrModel.transform(test_xy)

predictions.show(n=5, truncate=False)

In [0]:
predictionAndLabels = predictions.select("prediction", "label").rdd
metrics = MulticlassMetrics(predictionAndLabels)


print(f"accuracy: {metrics.accuracy}")
print(f"precision: {metrics.precision(1.0)}")

print(f"recall: {metrics.recall(1.0)}")

print(f"f1 score: {metrics.fMeasure(1.0, 1.0)}")